# Part 2 - Patient similarity networks

In [2]:
# standard libraries
import os
import pickle

# scientific and data manipulation libraries
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.feature_selection import mutual_info_regression
from astropy.stats import median_absolute_deviation
import mygene
import astropy

# graph and network libraries
import networkx as nx
from pyvis.network import Network

# visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.io as pio

- Based on the same expression matrix we can create a patient similarity network.
- Transposing the matrix will switch the rows and columns,
- meaning that patients will become the columns instead of genes
- By doing this, you can compute the correlation (or similarity) between patients based on their gene expression profiles,
- and then create a network where nodes represent patients and edges represent similarities.

In [4]:
# main data directory for the project

data_dir = '../../ismb_data'

In [4]:
# read in os.path.join(data_dir,"expression_data_filtered.csv")
df_renamed = pd.read_csv(os.path.join(data_dir,"expression_data_filtered.csv"),index_col=0)

In [5]:
# Transpose the matrix
patient_gene_matrix = df_renamed.T

In [6]:
patient_gene_matrix

,TCGA-38-7271,TCGA-55-7914,TCGA-95-7043,TCGA-73-4658,TCGA-86-8076,TCGA-55-7726,TCGA-44-6147,TCGA-50-5932,TCGA-44-2661,TCGA-86-7954,...,TCGA-97-A4M7,TCGA-62-A46R,TCGA-50-5055,TCGA-38-4628,TCGA-86-7713,TCGA-86-8073,TCGA-MN-A4N4,TCGA-53-7626,TCGA-44-A47G,TCGA-55-6969
A2M,17.7492,14.8513,14.1691,16.7238,15.6783,14.7566,16.4368,15.5476,15.5478,15.1337,...,15.9553,13.9511,16.3097,14.3934,15.8254,16.3773,14.9411,16.7343,15.6622,14.8136
A2ML1,4.4411,4.4530,4.5026,3.1704,4.7422,6.0918,4.0602,3.0827,3.4608,3.6367,...,4.7076,2.4821,8.0810,3.3415,5.3379,4.0817,6.7210,5.2575,3.7345,4.3058
A4GALT,10.1862,8.9312,9.0834,9.1443,9.5150,11.4452,9.1462,7.3597,9.1602,8.9344,...,9.1019,8.6653,10.8837,8.6304,9.3827,9.4234,11.9107,9.5379,9.8660,9.7702
AACSP1,3.5845,3.6762,3.8623,2.4821,3.2271,2.4821,5.6614,3.0827,3.9428,4.7089,...,5.4947,3.0956,2.4821,3.7710,2.9397,6.6519,8.8283,2.4821,4.1045,6.8425
AADAC,9.6415,6.5685,5.5634,4.3206,7.2321,6.8821,8.1176,4.7935,11.7750,7.0644,...,9.0856,3.8162,5.6471,4.6785,4.6578,6.7641,3.4947,5.7120,6.2214,5.2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZSCAN23,5.7385,4.3796,5.6728,3.6533,5.0478,6.0489,5.8058,3.7897,4.3451,5.5259,...,4.7076,2.4821,4.6683,3.4697,4.3516,7.2074,6.9167,6.0927,5.3854,3.6792
ZSCAN31,9.0817,10.9565,10.8095,8.9874,9.5810,9.0185,10.2085,9.8803,11.0327,10.2707,...,9.2033,10.5020,8.8073,9.8672,10.6419,10.2410,9.4672,10.0507,7.4843,8.5268
ZSCAN4,3.2709,3.0918,2.4821,3.4468,3.0118,5.0894,5.5341,3.5082,5.4987,3.3645,...,4.4088,7.4937,4.3167,5.0907,6.7676,5.1126,3.3143,4.2816,4.4650,2.4821
ZWINT,9.2615,10.1334,10.9149,9.7828,8.6640,11.1212,9.0718,10.7305,9.0013,10.4268,...,9.7614,10.7593,9.7087,10.4729,10.9794,10.1390,10.0583,9.4627,9.5817,10.8228


In [7]:
# Dictionary to store different correlation matrices
patient_correlation_matrices = {}

# Pearson correlation
patient_correlation_matrices['pearson'] = patient_gene_matrix.corr(method='pearson')

# patient_correlation_matrices['biweight_midcorrelation'] = abs_bicorr(patient_gene_matrix)

In [17]:
def create_graph_from_correlation(correlation_matrix, threshold=0.8):
    """
    Creates a graph from a correlation matrix using a specified threshold.

    Parameters:
    correlation_matrix (pd.DataFrame): DataFrame containing the correlation matrix.
    threshold (float): Threshold for including edges based on correlation value.

    Returns:
    G (nx.Graph): Graph created from the correlation matrix.
    """
    G = nx.Graph()

    # Add nodes
    for node in correlation_matrix.columns:
        G.add_node(node)

    # Add edges with weights above the threshold
    for i in range(correlation_matrix.shape[0]):
        for j in range(i + 1, correlation_matrix.shape[1]):
            if i != j:  # Ignore the diagonal elements
                weight = correlation_matrix.iloc[i, j]
                if abs(weight) >= threshold:
                    G.add_edge(correlation_matrix.index[i], correlation_matrix.columns[j], weight=weight)

    return G


def clean_graph(G, degree_threshold=1, keep_largest_component=True):
    """
    Cleans the graph by performing several cleaning steps:
    - Removes unconnected nodes (isolates)
    - Removes self-loops
    - Removes nodes with a degree below a specified threshold
    - Keeps only the largest connected component (optional)

    Parameters:
    G (nx.Graph): The NetworkX graph to clean.
    degree_threshold (int): Minimum degree for nodes to keep.
    keep_largest_component (bool): Whether to keep only the largest connected component.

    Returns:
    G (nx.Graph): Cleaned graph.
    """
    G = G.copy()  # Work on a copy of the graph to avoid modifying the original graph

    # Remove self-loops
    G.remove_edges_from(nx.selfloop_edges(G))

    # Remove nodes with no edges (isolates)
    G.remove_nodes_from(list(nx.isolates(G)))

    # Remove nodes with degree below the threshold
    low_degree_nodes = [node for node, degree in dict(G.degree()).items() if degree < degree_threshold]
    G.remove_nodes_from(low_degree_nodes)

    # Keep only the largest connected component
    if keep_largest_component:
        largest_cc = max(nx.connected_components(G), key=len)
        G = G.subgraph(largest_cc).copy()

    return G

def print_graph_info(G):
    """
    Prints basic information about the graph.
    
    Parameters:
    G (nx.Graph): The NetworkX graph.
    """
    print(f"Number of nodes: {G.number_of_nodes()}")
    print(f"Number of edges: {G.number_of_edges()}")
    print("Sample nodes:", list(G.nodes)[:10])  # Print first 10 nodes as a sample
    print("Sample edges:", list(G.edges(data=True))[:10])  # Print first 10 edges as a sample
    
    # Check for self-loops
    self_loops = list(nx.selfloop_edges(G))
    if self_loops:
        print(f"Number of self-loops: {len(self_loops)}")
        print("Self-loops:", self_loops)
    else:
        print("No self-loops in the graph.")

def calculate_average_clustering(G):
    """
    Calculates and prints the average clustering coefficient of the graph.
    
    Parameters:
    G (nx.Graph): The NetworkX graph.
    """
    avg_clustering = nx.average_clustering(G)
    print(f"Average clustering coefficient: {avg_clustering}")





def knn_sparsification(graph, k):
    """
    Sparsifies the graph by keeping only the top-k edges with the highest weights for each node.

    Parameters:
    graph (nx.Graph): The original NetworkX graph.
    k (int): The number of nearest neighbors to keep for each node.

    Returns:
    nx.Graph: The sparsified graph.
    """
    graph_copy = graph.copy()
    sparsified_graph = nx.Graph()
    sparsified_graph.add_nodes_from(graph_copy.nodes(data=True))
    
    for node in graph_copy.nodes():
        edges = sorted(graph_copy.edges(node, data=True), key=lambda x: x[2].get('weight', 0), reverse=True)
        sparsified_graph.add_edges_from(edges[:k])
    
    return sparsified_graph

In [ ]:
patient_pearson_graph = create_graph_from_correlation(patient_correlation_matrices['pearson'], threshold=0.8)

In [ ]:
visualize_graph(patient_pearson_graph, title='Pearson Correlation Network (Threshold = 0.8)')

In [ ]:
# Clean the graph by removing unconnected nodes
patient_pearson_graph_pruned = clean_graph(patient_pearson_graph,
                                    degree_threshold=1,
                                    keep_largest_component=True)

In [ ]:
visualize_graph(patient_pearson_graph_pruned, title='Pearson Patient Correlation Network (Threshold = 0.8)')

In [ ]:
patient_pearson_graph_pruned_knn = knn_sparsification(patient_pearson_graph_pruned, k=10)

In [ ]:
print_graph_info(patient_pearson_graph_pruned)

In [ ]:
print_graph_info(patient_pearson_graph_pruned_knn)

In [ ]:
visualize_graph(patient_pearson_graph_pruned_knn, title='K-Nearest Neighbors (k=10) Patient Correlation Network')

In [ ]:
nx.write_gml(patient_pearson_graph_pruned_knn, os.path.join(data_dir,'patient_coexpression_network_knn.gml'))

# DNA methylation based Patient Similarity Network

## Task

In the second task, we are preparing an additional network for the same patients, this time based on DNA methylation data.

In [18]:
# Load the data using pickle
with open(os.path.join(data_dir,"ISMB_TCGA_DNAm.pkl") , 'rb') as file : 
    data = pd.read_pickle(file)

meth_data = data["datExpr"]
meth_data


,cg23057992,cg16602369,cg20545544,cg04571941,cg23091104,cg11738485,cg17735539,cg12662576,cg10554839,cg08866780,...,cg04339424,cg01318691,cg27068965,cg23028848,cg21122529,cg22254072,cg10237911,cg09874052,cg27624178,cg02857943
TCGA-55-7914,0.039144,0.078881,0.032201,0.098520,0.971626,0.020020,0.929670,0.033325,0.043767,0.028051,...,0.053027,0.046047,0.951328,0.045767,0.048185,0.014262,0.034541,0.034967,0.041538,0.053116
TCGA-38-4631,0.954805,0.047287,0.027976,0.925794,0.144636,0.365842,0.156414,0.027917,0.047609,0.049901,...,0.048398,0.047221,0.939891,0.056141,0.047459,0.014548,0.056591,0.039086,0.044946,0.055473
TCGA-73-4658,0.966071,0.126549,0.030316,0.945166,0.976080,0.975638,0.942018,0.066659,0.052100,0.047054,...,0.040815,0.042040,0.951235,0.045477,0.074037,0.019053,0.029978,0.039345,0.060650,0.041385
TCGA-50-5932,0.965140,0.061194,0.935457,0.923421,0.965443,0.027598,0.040885,0.036178,0.045745,0.046439,...,0.057840,0.045106,0.948255,0.044558,0.039484,0.013837,0.037048,0.032503,0.058576,0.037666
TCGA-55-7576,0.958612,0.071607,0.027077,0.940574,0.976870,0.785073,0.953674,0.945107,0.056088,0.955472,...,0.046921,0.051442,0.954000,0.034352,0.031497,0.014924,0.035520,0.043547,0.056388,0.052208
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-50-8457,0.960316,0.940219,0.046927,0.942980,0.977751,0.633731,0.940723,0.053398,0.059292,0.968875,...,0.047640,0.062680,0.954824,0.043137,0.048599,0.014066,0.031109,0.039247,0.035110,0.054273
TCGA-91-6840,0.964250,0.921250,0.957943,0.958416,0.981439,0.278432,0.968525,0.032143,0.083996,0.037912,...,0.045197,0.066271,0.963420,0.047621,0.060854,0.015823,0.045674,0.040006,0.052008,0.040826
TCGA-86-8280,0.948855,0.942925,0.958506,0.945667,0.970511,0.544447,0.921924,0.939427,0.082121,0.045036,...,0.062367,0.058898,0.954355,0.045702,0.048930,0.019321,0.039341,0.036627,0.054269,0.056795
TCGA-95-A4VK,0.037255,0.050940,0.036565,0.098237,0.976780,0.683394,0.960073,0.025232,0.062487,0.026908,...,0.056248,0.053591,0.947020,0.035344,0.057783,0.010943,0.051868,0.035566,0.044910,0.042269


In [6]:
# load the data from the pickle file
with open(os.path.join(data_dir,"ISMB_TCGA_GE.pkl"), 'rb') as file:
    GE_data = pickle.load(file)


In [10]:
GE_data["datMeta"]["patient"].to_list()

['TCGA-38-7271',
 'TCGA-55-7914',
 'TCGA-95-7043',
 'TCGA-73-4658',
 'TCGA-86-8076',
 'TCGA-55-7726',
 'TCGA-44-6147',
 'TCGA-50-5932',
 'TCGA-44-2661',
 'TCGA-86-7954',
 'TCGA-73-4662',
 'TCGA-44-7671',
 'TCGA-78-8660',
 'TCGA-62-A46P',
 'TCGA-55-6978',
 'TCGA-50-6592',
 'TCGA-38-4625',
 'TCGA-80-5611',
 'TCGA-86-8054',
 'TCGA-55-6986',
 'TCGA-L9-A5IP',
 'TCGA-69-7764',
 'TCGA-49-6744',
 'TCGA-75-5125',
 'TCGA-38-4626',
 'TCGA-69-7763',
 'TCGA-86-8279',
 'TCGA-93-8067',
 'TCGA-97-8179',
 'TCGA-55-A48Y',
 'TCGA-86-8055',
 'TCGA-91-6835',
 'TCGA-55-6982',
 'TCGA-55-A4DF',
 'TCGA-44-6774',
 'TCGA-50-5066',
 'TCGA-05-5423',
 'TCGA-67-3774',
 'TCGA-97-A4M2',
 'TCGA-95-7567',
 'TCGA-49-AAR0',
 'TCGA-44-2656',
 'TCGA-53-7813',
 'TCGA-O1-A52J',
 'TCGA-35-4122',
 'TCGA-55-8092',
 'TCGA-49-6761',
 'TCGA-49-4507',
 'TCGA-55-7816',
 'TCGA-78-7145',
 'TCGA-55-6983',
 'TCGA-53-7624',
 'TCGA-97-A4M3',
 'TCGA-50-5068',
 'TCGA-78-8648',
 'TCGA-44-6778',
 'TCGA-80-5608',
 'TCGA-86-8281',
 'TCGA-MP-A4T6

1. Load the smoking-related DNA methylation data from a tab-separated values (TSV) file. (os.path.join(data_dir,"smoking.tsv"))
2. Identify CpG sites that are commonly annotated in the smoking dataset
3. Filter the DNA methylation data to include only the common CpG sites identified in the previous step
4. Identify patients that are present in both the gene expression dataset and the methylation dataset.
5. Filter the methylation data to include only the common patients and common CpG sites.
6. Transpose the filtered methylation data matrix.

In [11]:
smoking_df = pd.read_csv(os.path.join(data_dir,"smoking.tsv") , delimiter='\t') #Tab seperated document from EWAS
common_annotated_cpgs = list(smoking_df['cpg'].value_counts()[smoking_df['cpg'].value_counts() > 10].index) 
cpgs = set(common_annotated_cpgs) & set(meth_data.columns)


common_patients = set(GE_data["datMeta"]["patient"].to_list()) & set(meth_data.index)


meth_data_filt = meth_data.loc[ list(common_patients) , list(cpgs)]

patient_meth_matrix = meth_data_filt.T


In [13]:
patient_meth_matrix 


,TCGA-L9-A5IP,TCGA-97-8177,TCGA-97-A4M5,TCGA-NJ-A7XG,TCGA-49-4487,TCGA-80-5608,TCGA-50-5045,TCGA-78-7536,TCGA-50-8457,TCGA-78-7150,...,TCGA-91-8497,TCGA-55-7576,TCGA-50-5935,TCGA-55-7726,TCGA-44-7659,TCGA-67-6216,TCGA-73-4676,TCGA-44-6779,TCGA-50-5066,TCGA-44-A47A
cg06235438,0.704462,0.765600,0.735481,0.889260,0.626550,0.796208,0.691311,0.902062,0.739429,0.875700,...,0.787259,0.790955,0.801376,0.797787,0.781594,0.766541,0.595076,0.729774,0.751685,0.581357
cg16519923,0.350686,0.613381,0.530897,0.564738,0.536597,0.689610,0.507226,0.769669,0.547481,0.546939,...,0.494253,0.653185,0.603005,0.462919,0.500837,0.660526,0.393625,0.610697,0.567744,0.382835
cg03636183,0.558301,0.522365,0.590719,0.542055,0.650863,0.588275,0.619079,0.446724,0.580573,0.391529,...,0.540174,0.640841,0.651893,0.465395,0.690399,0.632989,0.463317,0.594800,0.564197,0.653906
cg16556677,0.643590,0.733000,0.637958,0.588998,0.653712,0.668136,0.669321,0.546550,0.691405,0.547689,...,0.633321,0.784905,0.772402,0.521192,0.752280,0.838290,0.575633,0.793454,0.659951,0.824979
cg24090911,0.679264,0.756709,0.761862,0.789242,0.784405,0.761720,0.709197,0.533741,0.812883,0.767221,...,0.577648,0.870502,0.917933,0.726192,0.864028,0.829835,0.496581,0.874250,0.681626,0.842857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cg23161492,0.554244,0.560400,0.409177,0.719385,0.449251,0.486408,0.313101,0.540278,0.474049,0.657513,...,0.429354,0.584473,0.373168,0.470337,0.699737,0.265069,0.422142,0.515816,0.354033,0.661620
cg04180046,0.760798,0.627530,0.561161,0.668213,0.710857,0.682087,0.712194,0.728562,0.691135,0.266435,...,0.508344,0.791685,0.576055,0.372007,0.731834,0.580984,0.422930,0.878109,0.697204,0.759649
cg04885881,0.526381,0.287194,0.449110,0.294257,0.389154,0.383441,0.279278,0.329857,0.229649,0.496855,...,0.289696,0.354345,0.330573,0.307197,0.246585,0.238996,0.523181,0.471247,0.349305,0.284987
cg24556382,0.481587,0.726927,0.635479,0.595553,0.679223,0.465760,0.571945,0.215526,0.660404,0.345572,...,0.576382,0.558424,0.812648,0.330948,0.661905,0.398924,0.873810,0.411682,0.612189,0.892020


Calculate the pearson correlation of the data in a similar fashion.

In [14]:
# Dictionary to store different correlation matrices
p_meth_correlation_matrices = {}

# Pearson correlation
p_meth_correlation_matrices['pearson'] = patient_meth_matrix.corr(method='pearson')


In [15]:
p_meth_correlation_matrices['pearson']

,TCGA-L9-A5IP,TCGA-97-8177,TCGA-97-A4M5,TCGA-NJ-A7XG,TCGA-49-4487,TCGA-80-5608,TCGA-50-5045,TCGA-78-7536,TCGA-50-8457,TCGA-78-7150,...,TCGA-91-8497,TCGA-55-7576,TCGA-50-5935,TCGA-55-7726,TCGA-44-7659,TCGA-67-6216,TCGA-73-4676,TCGA-44-6779,TCGA-50-5066,TCGA-44-A47A
TCGA-L9-A5IP,1.000000,0.750352,0.792929,0.775113,0.771453,0.762132,0.792639,0.700945,0.777697,0.749786,...,0.693956,0.745302,0.702926,0.806156,0.696502,0.681261,0.706900,0.815488,0.824247,0.662803
TCGA-97-8177,0.750352,1.000000,0.906620,0.872505,0.896966,0.857012,0.887044,0.742459,0.933424,0.746607,...,0.910966,0.873954,0.879528,0.827868,0.863407,0.862244,0.823589,0.843569,0.870039,0.855682
TCGA-97-A4M5,0.792929,0.906620,1.000000,0.875438,0.900053,0.862816,0.901533,0.742855,0.913055,0.784530,...,0.904910,0.822555,0.865804,0.843548,0.841370,0.811997,0.910897,0.781393,0.870607,0.771044
TCGA-NJ-A7XG,0.775113,0.872505,0.875438,1.000000,0.813323,0.825825,0.840628,0.741381,0.908135,0.725394,...,0.872843,0.868354,0.884608,0.837838,0.900364,0.836612,0.753218,0.812014,0.832514,0.793274
TCGA-49-4487,0.771453,0.896966,0.900053,0.813323,1.000000,0.851447,0.926583,0.760655,0.892917,0.718079,...,0.858880,0.831646,0.831506,0.769613,0.821722,0.843301,0.826595,0.809863,0.901425,0.812857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-67-6216,0.681261,0.862244,0.811997,0.836612,0.843301,0.756288,0.844520,0.623768,0.849560,0.629313,...,0.827494,0.888694,0.839266,0.761374,0.887381,1.000000,0.675235,0.829117,0.827366,0.855497
TCGA-73-4676,0.706900,0.823589,0.910897,0.753218,0.826595,0.803814,0.827674,0.694755,0.814776,0.736933,...,0.829156,0.678059,0.779263,0.765433,0.711141,0.675235,1.000000,0.633899,0.802940,0.675955
TCGA-44-6779,0.815488,0.843569,0.781393,0.812014,0.809863,0.777771,0.837654,0.667718,0.814770,0.694002,...,0.735527,0.861013,0.758416,0.807886,0.781137,0.829117,0.633899,1.000000,0.859323,0.767080
TCGA-50-5066,0.824247,0.870039,0.870607,0.832514,0.901425,0.849570,0.948990,0.756705,0.873864,0.781976,...,0.807555,0.864433,0.809323,0.854363,0.795254,0.827366,0.802940,0.859323,1.000000,0.751277


In [16]:
p_meth_pearson_graph = create_graph_from_correlation(p_meth_correlation_matrices['pearson'], threshold=0.8)
# Clean the graph by removing unconnected nodes
p_meth_pearson_graph_pruned = clean_graph(p_meth_pearson_graph,
                                    degree_threshold=1,
                                    keep_largest_component=True)

visualize_graph(p_meth_pearson_graph_pruned, title='Pearson Correlation Network (Threshold = 0.8)')


NameError: name 'create_graph_from_correlation' is not defined

In [ ]:
p_meth_pearson_graph_pruned_knn = knn_sparsification(p_meth_pearson_graph_pruned, k=10)


In [ ]:
visualize_graph(p_meth_pearson_graph_pruned_knn, title='Pearson Correlation Network (Threshold = 0.8)')


In [ ]:
nx.write_gml(p_meth_pearson_graph_pruned_knn, os.path.join(data_dir,'patient_meth_network_knn.gml'))

In [ ]:
def save_graphs(correlation_matrices_dict, data_dir, matrix_names):
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    
    # Define parameter combinations
    thresholds = [0.6, 0.7, 0.8, 0.9]
    k_values = [3, 5, 7, 10]
    degree_thresholds = [0, 1]
    keep_largest_component_options = [False, True]

    for matrix_name in matrix_names:
        if matrix_name not in correlation_matrices_dict:
            print(f"Matrix {matrix_name} not found in the provided dictionary. Skipping...")
            continue
        
        correlation_matrix = correlation_matrices_dict[matrix_name]
        
        for threshold in thresholds:
            # Create the initial graph from the correlation matrix
            initial_graph = create_graph_from_correlation(correlation_matrix, threshold=threshold)
            
            for degree_threshold in degree_thresholds:
                for keep_largest_component in keep_largest_component_options:
                    # Clean the graph
                    cleaned_graph = clean_graph(initial_graph, degree_threshold=degree_threshold, keep_largest_component=keep_largest_component)
                    
                    for k in k_values:
                        # Sparsify the graph using KNN
                        sparsified_graph = knn_sparsification(cleaned_graph, k)
                        
                        # Calculate the average clustering coefficient
                        avg_clustering = nx.average_clustering(sparsified_graph)
                        
                        # Construct the filename with relevant information
                        filename = f"{matrix_name}_network_threshold_{threshold}_k_{k}_degree_{degree_threshold}_largest_{keep_largest_component}_clustering_{avg_clustering:.2f}.gml"
                        filepath = os.path.join(data_dir, filename)
                        
                        # Save the graph to a GML file
                        nx.write_gml(sparsified_graph, filepath)
                        
                        # Print the details
                        print(f"Saved: {filepath}")

def main(correlation_matrices_dict, matrix_names):
    data_dir = '../../ismb_data/05072024/PSN-Met'  # Adjust the directory path as needed
    save_graphs(correlation_matrices_dict, data_dir, matrix_names)

if __name__ == "__main__":
    # correlation_matrices_dict = {
    #     'pearson': pd.DataFrame(),  # Replace with actual data
    #     'spearman': pd.DataFrame()  # Replace with actual data if needed
    # }
    matrix_names = ['pearson']  # Replace with the desired correlation matrix names
    main(p_meth_correlation_matrices, matrix_names)